In [ ]:
import json
import pandas as pd
import copy

aggregator_file = "./aggregate_scores_512.json"
scores = json.load(open(aggregator_file))

In [ ]:
len(scores)

In [ ]:
results = []

num_classes = 4

def is_same_experiment(e1, e2):
    return e1['is_normalized'] == e2['is_normalized'] and e1['loss_type'] == e2['loss_type']

def experiment_exists(exp):
    exists = False

    for r in results:
        if is_same_experiment(exp, r):
            exists = True

    return exists

# Each outer loop = each k
for k in range(len(scores)):
    for experiment in scores[k]:
        if not experiment_exists(experiment['experiment']):
            result_obj = copy.deepcopy(experiment['experiment'])
            result_obj['ave_macro_score'] = 0

            for i in range(num_classes):
                result_obj['class_{}'.format(i)] = 0

            results.append(result_obj)

for result_obj_index in range(len(results)):
    for k in range(len(scores)):
        for exp_obj in scores[k]:
            result_obj = results[result_obj_index]
            if is_same_experiment(exp_obj['experiment'], result_obj):
                results[result_obj_index]['ave_macro_score'] += exp_obj['ave_macro_score']

                for label_index in range(num_classes):
                    ave_score = 0
                    for label_score_set in  exp_obj['label_scores']:
                        if label_index > len(label_score_set) - 1:
                            ave_score += 1
                        else:
                            ave_score += label_score_set[label_index]

                    ave_score /= len(exp_obj['label_scores'])
                    results[result_obj_index]['class_{}'.format(label_index)] += ave_score

# Average out
for result_obj in results:
    result_obj['ave_macro_score'] = result_obj['ave_macro_score'] / len(scores)

    for label_index in range(num_classes):
        result_obj['class_{}'.format(label_index)] /= len(scores)

results

In [ ]:
import pandas as pd

model_types = []

for result in results:
    model_types.append("{}-{}".format(result['loss_type'], result['is_normalized']))

df = pd.DataFrame(results, index=model_types)

df

In [ ]:
ax = df.plot.bar(rot=0)

In [ ]:
best_models = []

metrics = [
    'ave_macro_score',
    'class_0',
    'class_1',
    'class_2',
    'class_3'
]

for metric in metrics:
    best_score = 0.001
    best_model = ''
    improvement_raw = 0.0
    for result in results:
        if result[metric] > best_score:
            improvement_raw = result[metric] - best_score
            best_score = result[metric]
            best_model = "{}-{}".format(result['loss_type'], result['is_normalized'])

    best_models.append({'metric': metric, 'best_model': best_model, 'improvement_raw': improvement_raw })

df_best_models = pd.DataFrame(best_models)

df_best_models